In [97]:
import vectorbt as vbt
import numpy as np
import pandas as pd
import datetime
import plotly.express as px
from xbbg import blp
import os
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
import logging

# Import custom modules with an alias
import bloomberg_data as bd
import transformations as tr
import visuals as vis




In [98]:

# Main data retrieval and merging process
tickers = ['.MIDERCAD U Index', '.CADIG F Index', 'VIX Index', '.HYUSER U Index', '.IGUSER U Index','SPX INDEX']
fields = [['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST'], ['PX_LAST']]
start_date = '2000-01-01'
end_date = '2025-12-31'
column_names = [['cad_ig_er_index'], ['cad_ig_sprds'], ['vix'], ['us_hy_er_index'], ['us_ig_er_index'],['spx_index']]
frequency = 'M'  # Single frequency for all tickers

dataframes = []

for ticker, field, col_name in zip(tickers, fields, column_names):
    df = bd.get_single_ticker_data(ticker, field, start_date, end_date, freq=frequency, column_names=col_name)
    dataframes.append(df)
    logging.info(f"Data for {ticker}:")
    logging.info(df.head())  # Print the first few rows of each dataframe

# Merge all dataframes
merged_data = bd.merge_dataframes(dataframes, method='outer')

# Print the final merged data and its information
logging.info("Merged data head:")
logging.info(merged_data.head())
logging.info('----------------------------------------------------------------')
logging.info('----------------------------------------------------------------')
logging.info(merged_data.tail())
logging.info(merged_data.info())

# Rename the index to "Date" and reset it
merged_data.index.name = 'Date'
csv_data = merged_data.reset_index()

# Save the dataframe to a CSV file
csv_data.to_csv('Outputs/csv_data.csv', index=False)

# Rename for further use
data = merged_data

2024-07-09 14:44:00,976 - INFO - Retrieving data for ticker: .MIDERCAD U Index with frequency: MONTHLY
2024-07-09 14:44:01,711 - INFO - Retrieved data shape for .MIDERCAD U Index: (261, 1)
2024-07-09 14:44:01,711 - INFO - Cleaned data shape for .MIDERCAD U Index: (261, 1)
2024-07-09 14:44:01,712 - INFO - Successfully retrieved data for ticker: .MIDERCAD U Index
2024-07-09 14:44:01,712 - INFO - Data for .MIDERCAD U Index:
2024-07-09 14:44:01,713 - INFO -             cad_ig_er_index
2002-11-29           1.0143
2002-12-31           1.0146
2003-01-31           1.0155
2003-02-28           1.0159
2003-03-31           1.0142
2024-07-09 14:44:01,714 - INFO - Retrieving data for ticker: .CADIG F Index with frequency: MONTHLY
2024-07-09 14:44:02,243 - INFO - Retrieved data shape for .CADIG F Index: (262, 1)
2024-07-09 14:44:02,244 - INFO - Cleaned data shape for .CADIG F Index: (262, 1)
2024-07-09 14:44:02,245 - INFO - Successfully retrieved data for ticker: .CADIG F Index
2024-07-09 14:44:02,24

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 295 entries, 2000-01-31 to 2024-07-31
Freq: BM
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   cad_ig_er_index  295 non-null    float64
 1   cad_ig_sprds     295 non-null    float64
 2   vix              295 non-null    float64
 3   us_hy_er_index   295 non-null    float64
 4   us_ig_er_index   295 non-null    float64
 5   spx_index        295 non-null    float64
dtypes: float64(6)
memory usage: 16.1 KB


In [123]:
import pandas as pd
import vectorbt as vbt

# Assuming your data is already loaded into a DataFrame named 'data'

# Define the date range for backtesting
start_date = '2000-01-01'
end_date = '2024-06-28'

# Get the actual start and end dates from the data
actual_start_date = data.index.min()
actual_end_date = data.index.max()

# Adjust start_date and end_date to be within the actual data range
start_date = max(pd.Timestamp(start_date), actual_start_date)
end_date = min(pd.Timestamp(end_date), actual_end_date)

# Filter data based on the adjusted date range
data_range = data[(data.index >= start_date) & (data.index <= end_date)].copy()

print(f"Using data from {start_date.date()} to {end_date.date()}")

# Create the 3-period moving average
data_range['SMA_3'] = data_range['cad_ig_er_index'].rolling(window=3).mean()

# Generate signals
data_range['buy_signal'] = data_range['cad_ig_er_index'] > data_range['SMA_3']
data_range['sell_signal'] = data_range['cad_ig_er_index'] < data_range['SMA_3']

# Ensure the strategy starts invested
data_range.loc[data_range.index[0], 'buy_signal'] = True

# Convert signals to vectorbt format
entries = data_range['buy_signal']
exits = data_range['sell_signal']

# Run the backtest for the strategy
pf_3ma_cad_ig = vbt.Portfolio.from_signals(
    close=data_range['cad_ig_er_index'],
    entries=entries,
    exits=exits,
    freq='30D',  # Approximating monthly frequency
    init_cash=1000000,  # Set initial cash amount
    fees=0.001  # Set transaction fees (0.1% in this example)
)

# Create a buy and hold portfolio
pf_buy_hold = vbt.Portfolio.from_holding(
    close=data_range['cad_ig_er_index'],
    freq='30D',
    init_cash=1000000,  # Set initial cash amount
    fees=0.001  # Set transaction fees (0.1% in this example)
)

# Print performance stats for the strategy
print("\nStrategy Performance Stats:")
print(pf_3ma_cad_ig.stats())

# Print performance stats for the buy and hold portfolio
print("\nBuy and Hold Performance Stats:")
print(pf_buy_hold.stats())

# Print annualized returns for both
strategy_annualized_return = pf_3ma_cad_ig.annualized_return() * 100
buy_hold_annualized_return = pf_buy_hold.annualized_return() * 100

print(f"\nStrategy Annualized Return: {strategy_annualized_return:.2f}%")
print(f"Buy and Hold Annualized Return: {buy_hold_annualized_return:.2f}%")

# Uncomment the following line if you want to see available subplots
# print("Available subplots:", pf_3ma_cad_ig.subplots)

# print("Available subplots:", pf_3ma_cad_ig.subplots)


Using data from 2000-01-31 to 2024-06-28

Strategy Performance Stats:
Start                                 2000-01-31 00:00:00
End                                   2024-06-28 00:00:00
Period                                 8820 days 00:00:00
Start Value                                     1000000.0
End Value                                  1595637.538891
Total Return [%]                                59.563754
Benchmark Return [%]                            37.286799
Max Gross Exposure [%]                              100.0
Total Fees Paid                              86704.195241
Max Drawdown [%]                                 3.112971
Max Drawdown Duration                   930 days 00:00:00
Total Trades                                           33
Total Closed Trades                                    33
Total Open Trades                                       0
Open Trade PnL                                        0.0
Win Rate [%]                                    57.575758
Be

In [124]:
import vectorbt as vbt
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Assuming pf_3ma_cad_ig is your Portfolio object

# Define the order of subplots based on logical grouping
subplot_order = [
    'cum_returns', 'drawdowns', 'underwater',
    'value', 'asset_value', 'cash',
    'assets', 'asset_flow', 'cash_flow',
    'gross_exposure', 'net_exposure',
    'orders'  # Added orders to the subplot list
]

# Create a list of subplots that are actually available in the Portfolio object
available_subplots = [subplot for subplot in subplot_order if subplot in pf_3ma_cad_ig.subplots]

# Calculate the number of rows needed (2 columns)
num_rows = (len(available_subplots) + 1) // 2

# Create the main figure
fig = make_subplots(
    rows=num_rows,
    cols=2,
    subplot_titles=[pf_3ma_cad_ig.subplots[subplot]['title'] for subplot in available_subplots],
    vertical_spacing=0.1,
    horizontal_spacing=0.05
)

# Add each subplot to the figure
for i, subplot in enumerate(available_subplots):
    row = i // 2 + 1
    col = i % 2 + 1
    
    # Get the plot for the current subplot
    plot_func = pf_3ma_cad_ig.subplots[subplot]['plot_func']
    
    if '.' in plot_func:
        # Handle methods of sub-objects (like orders.plot)
        obj_name, method_name = plot_func.split('.')
        obj = getattr(pf_3ma_cad_ig, obj_name)
        subplot_fig = getattr(obj, method_name)()
    else:
        # Handle direct methods of the Portfolio object
        subplot_fig = getattr(pf_3ma_cad_ig, plot_func)()
    
    # Add the traces from the subplot to the main figure
    for trace in subplot_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    # Update the layout for this subplot
    fig.update_xaxes(title_text="Date", row=row, col=col)
    fig.update_yaxes(title_text=pf_3ma_cad_ig.subplots[subplot]['yaxis_kwargs']['title'], row=row, col=col)

# Update the overall layout
fig.update_layout(
    title='Comprehensive Portfolio Analysis',
    height=350 * num_rows,  # Increased height for better visibility
    width=1200,
    showlegend=False
)

# Show the plot
fig.show()

# Print a summary of included subplots
print("Subplots included in the analysis:")
for subplot in available_subplots:
    print(f"- {pf_3ma_cad_ig.subplots[subplot]['title']}")

Subplots included in the analysis:
- Cumulative Returns
- Drawdowns
- Underwater
- Value
- Asset Value
- Cash
- Assets
- Asset Flow
- Cash Flow
- Gross Exposure
- Net Exposure
- Orders


In [130]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from itertools import product
import matplotlib.pyplot as plt

def calculate_drawdown(series):
    """Calculate drawdown for a given series."""
    peak = series.cummax()
    drawdown = (series - peak) / peak
    return drawdown

def run_strategy(data, sma_window, off_threshold, on_threshold):
    """Run the enhanced MA strategy with drawdown rules."""
    sma_window = int(sma_window)
    if sma_window <= 0:
        raise ValueError("SMA window must be a positive integer")

    data = data.copy()
    # Resample data to 30D frequency
    data_30d = data.resample('30D').last()
    
    data_30d['SMA'] = data_30d['cad_ig_er_index'].rolling(window=sma_window).mean()
    data_30d['buy_signal'] = data_30d['cad_ig_er_index'] > data_30d['SMA']
    data_30d['sell_signal'] = data_30d['cad_ig_er_index'] < data_30d['SMA']
    data_30d['drawdown'] = calculate_drawdown(data_30d['cad_ig_er_index'])
    
    strategy_active = True
    for i in range(1, len(data_30d)):
        if strategy_active and data_30d['drawdown'].iloc[i] <= off_threshold:
            strategy_active = False
        elif not strategy_active and data_30d['drawdown'].iloc[i] >= on_threshold:
            strategy_active = True
        
        if not strategy_active:
            data_30d.loc[data_30d.index[i], 'buy_signal'] = False
            data_30d.loc[data_30d.index[i], 'sell_signal'] = False
    
    data_30d.loc[data_30d.index[0], 'buy_signal'] = True
    
    portfolio = vbt.Portfolio.from_signals(
        close=data_30d['cad_ig_er_index'],
        entries=data_30d['buy_signal'],
        exits=data_30d['sell_signal'],
        freq='30D',
        init_cash=1000000,
        fees=0.001
    )
    
    return portfolio

# Assuming 'data' is your DataFrame with 'cad_ig_er_index' column and DatetimeIndex

# Define parameter ranges for optimization
sma_windows = [1, 2, 3, 4]  # Adjusted for 30D frequency
off_thresholds = np.arange(-0.30, -0.05, 0.01)
on_thresholds = np.arange(-0.25, 0, 0.01)

# Perform grid search
results = []
for sma, off, on in product(sma_windows, off_thresholds, on_thresholds):
    if on > off:
        try:
            portfolio = run_strategy(data, sma, off, on)
            total_return = portfolio.total_return()
            results.append((sma, off, on, total_return))
        except Exception as e:
            print(f"Error with parameters: SMA={sma}, Off={off}, On={on}. Error: {str(e)}")

results_df = pd.DataFrame(results, columns=['SMA_Window', 'Off_Threshold', 'On_Threshold', 'Total_Return'])
best_params = results_df.loc[results_df['Total_Return'].idxmax()]

print("Optimization Results:")
print(f"Best SMA Window: {best_params['SMA_Window']}")
print(f"Best Off Threshold: {best_params['Off_Threshold']:.2%}")
print(f"Best On Threshold: {best_params['On_Threshold']:.2%}")
print(f"Best Total Return: {best_params['Total_Return']:.2%}")

# Run the optimized strategy
optimized_portfolio = run_strategy(data, int(best_params['SMA_Window']), best_params['Off_Threshold'], best_params['On_Threshold'])

# Create buy and hold portfolio
data_30d = data.resample('30D').last()
buy_hold_portfolio = vbt.Portfolio.from_holding(data_30d['cad_ig_er_index'], init_cash=1000000, fees=0.001)

# Print statistics
print("\nOptimized Strategy Statistics:")
print(optimized_portfolio.stats())
print("\nBuy and Hold Statistics:")
print(buy_hold_portfolio.stats())



Optimization Results:
Best SMA Window: 4.0
Best Off Threshold: -30.00%
Best On Threshold: -25.00%
Best Total Return: 66.27%

Optimized Strategy Statistics:
Start                                 2000-01-31 00:00:00
End                                   2024-07-23 00:00:00
Period                                 8970 days 00:00:00
Start Value                                     1000000.0
End Value                                  1662667.672667
Total Return [%]                                66.266767
Benchmark Return [%]                            37.523415
Max Gross Exposure [%]                              100.0
Total Fees Paid                              60544.912816
Max Drawdown [%]                                 3.635608
Max Drawdown Duration                   780 days 00:00:00
Total Trades                                           24
Total Closed Trades                                    23
Total Open Trades                                       1
Open Trade PnL                  

In [131]:
buy_hold_portfolio.plot()

FigureWidget({
    'data': [{'legendgroup': '0',
              'line': {'color': '#1f77b4'},
              'name': 'Close',
              'showlegend': True,
              'type': 'scatter',
              'uid': 'ae1686d3-7c2a-42a5-9eba-175639b9f1ee',
              'x': array([datetime.datetime(2000, 1, 31, 0, 0),
                          datetime.datetime(2000, 3, 1, 0, 0),
                          datetime.datetime(2000, 3, 31, 0, 0), ...,
                          datetime.datetime(2024, 5, 24, 0, 0),
                          datetime.datetime(2024, 6, 23, 0, 0),
                          datetime.datetime(2024, 7, 23, 0, 0)], dtype=object),
              'xaxis': 'x',
              'y': array([1.0143,    nan, 1.0143, ..., 1.3949, 1.3925, 1.3949]),
              'yaxis': 'y'},
             {'customdata': array([[     0.        , 984916.69032929,    999.000999  ]]),
              'hovertemplate': ('Order Id: %{customdata[0]}<br>' ... '<br>Fees: %{customdata[2]:.6f}'),
            

In [133]:
import pandas as pd
import numpy as np
import vectorbt as vbt
from itertools import product
import matplotlib.pyplot as plt

def calculate_drawdown(series):
    """Calculate drawdown for a given series."""
    peak = series.cummax()
    drawdown = (series - peak) / peak
    return drawdown

def run_strategy(data, sma_window, off_threshold, on_threshold, debug=False):
    """Run the enhanced MA strategy with drawdown rules."""
    sma_window = int(sma_window)
    if sma_window <= 0:
        raise ValueError("SMA window must be a positive integer")

    data = data.copy()
    data_30d = data.resample('30D').last()
    
    data_30d['SMA'] = data_30d['cad_ig_er_index'].rolling(window=sma_window).mean()
    data_30d['buy_signal'] = data_30d['cad_ig_er_index'] > data_30d['SMA']
    data_30d['sell_signal'] = data_30d['cad_ig_er_index'] < data_30d['SMA']
    data_30d['drawdown'] = calculate_drawdown(data_30d['cad_ig_er_index'])
    
    strategy_active = True
    for i in range(1, len(data_30d)):
        if strategy_active and data_30d['drawdown'].iloc[i] <= off_threshold:
            strategy_active = False
            if debug:
                print(f"Strategy turned off at {data_30d.index[i]}, drawdown: {data_30d['drawdown'].iloc[i]:.2%}")
        elif not strategy_active and data_30d['drawdown'].iloc[i] >= on_threshold:
            strategy_active = True
            if debug:
                print(f"Strategy turned on at {data_30d.index[i]}, drawdown: {data_30d['drawdown'].iloc[i]:.2%}")
        
        if not strategy_active:
            data_30d.loc[data_30d.index[i], 'buy_signal'] = False
            data_30d.loc[data_30d.index[i], 'sell_signal'] = False
    
    data_30d.loc[data_30d.index[0], 'buy_signal'] = True
    
    portfolio = vbt.Portfolio.from_signals(
        close=data_30d['cad_ig_er_index'],
        entries=data_30d['buy_signal'],
        exits=data_30d['sell_signal'],
        freq='30D',
        init_cash=1000000,
        fees=0.001
    )
    
    return portfolio

# Assuming 'data' is your DataFrame with 'cad_ig_er_index' column and DatetimeIndex

# Calculate the historical maximum drawdown
historical_drawdown = calculate_drawdown(data['cad_ig_er_index']).min()
print(f"Historical maximum drawdown: {historical_drawdown:.2%}")

# Define parameter ranges for optimization
sma_windows = [1, 2, 3, 4]  # Adjusted for 30D frequency
off_thresholds = np.linspace(historical_drawdown, 0, 20)  # 20 evenly spaced points
on_thresholds = np.linspace(historical_drawdown, 0, 20)  # 20 evenly spaced points

# Perform grid search
results = []
for sma, off, on in product(sma_windows, off_thresholds, on_thresholds):
    if on > off:
        try:
            portfolio = run_strategy(data, sma, off, on)
            total_return = portfolio.total_return()
            results.append((sma, off, on, total_return))
        except Exception as e:
            print(f"Error with parameters: SMA={sma}, Off={off}, On={on}. Error: {str(e)}")

results_df = pd.DataFrame(results, columns=['SMA_Window', 'Off_Threshold', 'On_Threshold', 'Total_Return'])
best_params = results_df.loc[results_df['Total_Return'].idxmax()]

print("\nOptimization Results:")
print(f"Best SMA Window: {best_params['SMA_Window']}")
print(f"Best Off Threshold: {best_params['Off_Threshold']:.2%}")
print(f"Best On Threshold: {best_params['On_Threshold']:.2%}")
print(f"Best Total Return: {best_params['Total_Return']:.2%}")

# Run the optimized strategy with debug=True
optimized_portfolio = run_strategy(data, int(best_params['SMA_Window']), best_params['Off_Threshold'], best_params['On_Threshold'], debug=True)

# Create buy and hold portfolio
data_30d = data.resample('30D').last()
buy_hold_portfolio = vbt.Portfolio.from_holding(data_30d['cad_ig_er_index'], init_cash=1000000, fees=0.001)

# Print statistics
print("\nOptimized Strategy Statistics:")
print(optimized_portfolio.stats())
print("\nBuy and Hold Statistics:")
print(buy_hold_portfolio.stats())



Historical maximum drawdown: -20.85%

Optimization Results:
Best SMA Window: 4.0
Best Off Threshold: -20.85%
Best On Threshold: -19.76%
Best Total Return: 66.27%
Strategy turned off at 2008-12-14 00:00:00, drawdown: -20.85%
Strategy turned on at 2009-01-13 00:00:00, drawdown: -19.58%

Optimized Strategy Statistics:
Start                                 2000-01-31 00:00:00
End                                   2024-07-23 00:00:00
Period                                 8970 days 00:00:00
Start Value                                     1000000.0
End Value                                  1662667.672667
Total Return [%]                                66.266767
Benchmark Return [%]                            37.523415
Max Gross Exposure [%]                              100.0
Total Fees Paid                              60544.912816
Max Drawdown [%]                                 3.635608
Max Drawdown Duration                   780 days 00:00:00
Total Trades                                 